<center><img src="./imge/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 6.使用OpenCV在Python中进行颜色检测

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：6.使用OpenCV在Python进行颜色检测.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：使用OpenCV在Python中进行颜色检测<br>

## 1、 颜色空间转换

In [1]:
# -*- coding: utf-8 -*-
import cv2 

flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags)


['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGRA', 'COLOR_BAYER_BG2BGR_EA', 'COLOR_BAYER_BG2BGR_VNG', 'COLOR_BAYER_BG2GRAY', 'COLOR_BAYER_BG2RGB', 'COLOR_BAYER_BG2RGBA', 'COLOR_BAYER_BG2RGB_EA', 'COLOR_BAYER_BG2RGB_VNG', 'COLOR_BAYER_GB2BGR', 'COLOR_BAYER_GB2BGRA', 'COLOR_BAYER_GB2BGR_EA', 'COLOR_BAYER_GB2BGR_VNG', 'COLOR_BAYER_GB2GRAY', 'COLOR_BAYER_GB2RGB', 'COLOR_BAYER_GB2RGBA', 'COLOR_BAYER_GB2RGB_EA', 'COLOR_BAYER_GB2RGB_VNG', 'COLOR_BAYER_GR2BGR', 'COLOR_BAYER_GR2BGRA', 'COLOR_BAYER_GR2BGR_EA', 'COLOR_BAYER_GR2BGR_VNG', 'COLOR_BAYER_GR2GRAY', 'COLOR_BAYER_GR2RGB', 'COLOR_BAYER_GR2RGBA', 'COLOR_BAYER_GR2RGB_EA', 'COLOR_BAYER_GR2RGB_VNG', 'COLOR_BAYER_RG2BGR', 'COLOR_BAYER_RG2BGRA', 'COLOR_BAYER_RG2BGR_EA', 'COLOR_BAYER_RG2BGR_VNG', 'COLOR_BAYER_RG2GRAY', 'COLOR_BAYER_RG2RGB', 'COLOR_BAYER_RG2RGBA', 'COLOR_BAYER_RG2RGB_EA', 'COLOR_BAYER_RG2RGB_VNG', 'COLOR_BGR2BGR555', 'COLOR_BGR2BGR565', 'COLOR_BGR2BGRA', 'COLOR_BGR2GRAY', 'COLOR_BGR2HLS', 'COLOR_BGR2HLS_FULL', 'COLOR_BGR2HSV', 'COLOR_

### 1.1 把BGR转换为HSV颜色空间

In [2]:
import sys
import numpy as np
import cv2
 
print("Please enter blue:")
blue = input()
print("Please enter green:")
green = input()
print("Please enter red:")
red = input()
 
color = np.uint8([[[blue, green, red]]])
hsv_color = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
 
hue = hsv_color[0][0][0]
 
print("Lower bound is :")
print("[" + str(hue-10) + ", 100, 100]\n")
 
print("Upper bound is :"),
print("[" + str(hue + 10) + ", 255, 255]")


Please enter blue:


 23


Please enter green:


 24


Please enter red:


 24


Lower bound is :
[20, 100, 100]

Upper bound is :
[40, 255, 255]


### 1.2 OpenCV颜色检测

In [3]:
import cv2
import numpy as np
from jetcam.utils import bgr8_to_jpeg

# 创建显示控件
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
mask_image = widgets.Image(format='jpeg', width=640, height=480)
image = widgets.Image(format='jpeg', width=640, height=480)
# 放置一个水平容器，让图片水平放置
image_container = widgets.HBox([image,mask_image])
display(image_container)

In [4]:
# 1表示我们想要BGR中的图像
img = cv2.imread('./images/makerobo.jpg', 1)

# 将imag在每个轴上的大小调整为20%
img = cv2.resize(img, (0,0), fx=0.2, fy=0.2)
# 将BGR图像转换为HSV图像
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# NumPy来创建数组以保存上下范围
# dtype = np。“uint8”表示数据类型是8位整数

lower_range = np.array([24, 100, 100], dtype=np.uint8)
upper_range = np.array([44, 255, 255], dtype=np.uint8)

# 为图像创建一个遮罩
mask = cv2.inRange(hsv, lower_range, upper_range)

# 将遮罩和图像并排显示
mask_image.value = bgr8_to_jpeg(mask)
image.value = bgr8_to_jpeg(img)

## 2.OpenCV 中颜色物体追踪

In [5]:
# 导入必要的包
from collections import deque
import numpy as np
import argparse
import imutils
import cv2

In [6]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

In [7]:
# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [8]:
# 构造参数解析并解析参数
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video",help="path to the (optional) video file")
ap.add_argument("-b", "--buffer", type=int, default=64,help="max buffer size")
args = vars(ap.parse_args(args=[]))

In [9]:
# 定义“黄色对象”的上下边界
# (或“球”)中的HSV颜色空间，然后初始化
# 跟踪点列表

colorLower = (24, 100, 100)
colorUpper = (44, 255, 255)
pts = deque(maxlen=args["buffer"])

In [10]:
dispW=640
dispH=480
flip=4
#调用CSI摄像头的属性参数
camSet='nvarguscamerasrc !  video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink'

if not args.get("video", False):
    camera = cv2.VideoCapture(camSet)
else:
    camera = cv2.VideoCapture(args["video"])

In [11]:
# 创建显示控件
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
Frame = widgets.Image(format='jpeg', width=480, height=320)
display(Frame)


Image(value=b'', format='jpeg', height='320', width='480')

In [12]:
def Video_display():
    while True:
        (grabbed, frame) = camera.read()
        #如果我们在观看视频时没有抓取帧，然后我们就到了视频的结尾了
        if args.get("video") and not grabbed:
            break
        # 调整帧大小，倒转(垂直翻转180度)，
        # 模糊它，并转换为HSV颜色空间
        frame = imutils.resize(frame, width=600)
        #frame = imutils.rotate(frame, angle=180)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # construct a mask for the color "green", then perform
        # a series of dilations and erosions to remove any small
        # blobs left in the mask
        mask = cv2.inRange(hsv, colorLower, colorUpper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # find contours in the mask and initialize the current
        # (x, y) center of the ball
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
        center = None

        if len(cnts) > 0:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            if radius > 10:
                # draw the circle and centroid on the frame,
                # then update the list of tracked points
                cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

        pts.appendleft(center)

        for i in range(1, len(pts)):
            if pts[i - 1] is None or pts[i] is None:
                continue

            thickness = int(np.sqrt(args["buffer"] / float(i + 1)) * 2.5)
            cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

        #cv2.imshow("Frame", frame)
        Frame.value = bgr8_to_jpeg(frame)

    camera.release()

t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

In [9]:
# 结束线程
stop_thread(t)